In [1]:
import cv2
import numpy as np
import os

In [2]:
X = []
Y = []
 
d = {}
cnt = 0
for f in os.listdir(): # scan the current directory for files
    if f.endswith(".npy"): # only take .npy files
        imgs = np.load(f)  # load images as np array 
        print(imgs.shape,f) # display the shape of image array
        X.append(imgs) # append image to list
        labels = np.ones((imgs.shape[0]))*cnt # adding 0 and 1 as last column
        Y.append(labels) # adding labels
        d[cnt] = f[:-4] # only name, extension removed
        cnt +=1

(20, 100, 100, 3) arharva.npy
(20, 100, 100, 3) yash.npy


In [3]:
X = np.array(X)
Y = np.array(Y)

X = X.reshape((40,100,100,3)) 
Y = Y.reshape((40,))

print(X.shape, Y.shape)
print(d)

(40, 100, 100, 3) (40,)
{0: 'arharva', 1: 'yash'}


In [4]:
#load the classifier to match and classify iamges
detector = cv2.CascadeClassifier("Datasets/FaceCascade/templatedata.xml")

# get control of 0th camera attached to pc
camera = cv2.VideoCapture(0)


In [5]:
def dist(p1,p2):
    return np.sum((p1-p2)**2)**0.5

def KNN(X,Y,test_point, K=5):
    d = [] # will store distance 
    m = X.shape[0] # 100,2 -> total number of points
    
    #Step1 - find distance of testpoint from all points
    for i in range(0,m):
        current_dist = dist(test_point, X[i]) # find distance of testpoint from current point
        d.append((current_dist,Y[i])) # append distance along with pair into d
    
    #Step2 - sort all points according to distances
    d = sorted(d) # by default based on the first key of tupple
    
    #Step3 - Pick the first k points
    d = d[:K] # pick only the first k elements, can be varied
    
    #Step4 - Create a numpy array
    d = np.array(d) 
    d = d[:,1] # get all rows, but elements from only first column
    
    #Step5 - Find out unique classes and counts
    uniq, cnts = np.unique(d,return_counts=True) # uniq denotes (0 and 1) and cnts is array of individual instances 
    idx = np.argmax(cnts) # get the index of maximum element. the cnts variable contains two elements. one is zero and other is one. on the zeroth element, there are instances of one. on one it contains instances of one.
    
    #Step6 - Find the label with maximum count
    pred = uniq[idx] # now pred has 0 or 1 whichever is maximum
    
    #Step7 - Map and send data 
    return int(pred) # replace 0 or 1 with respective mapping



In [6]:
# start an infinite loop to get all images
while True:
    # start capturing frames
    b,img = camera.read()
    
    # if no image captured, skip loop
    if b==False:
        continue
    
    # after every round of checks, scale 1.5 times
    faces = detector.detectMultiScale(img, 1.5)

    # if no faces detected in frame, skip loop execution
    if(len(faces)==0):
        continue
        
    # iterate for face in faces
    for f in faces:
        f = faces[0] # get the first or most significant face
        x,y,w,h = f # capture x coordinate, y coordinate, width and height from f
        green = (0,255,0) # define color for box as green
        cv2.rectangle(img, (x,y), (x+w,y+h), green, 5 )  # draw rectangle around faced  
        crop_face = img[y:y+h,x:x+w] # get reduced image of face fro photo
        small_face = cv2.resize(crop_face,(100,100)) # resize the small_face to avoid misfits
        
        predicted_name = d[KNN(X,Y,small_face)] # predict names of people
        
        cv2.putText(img,predicted_name,(x, y-20),cv2.FONT_HERSHEY_SIMPLEX, 1.0,(255,255,255)) # write names of people on image
        
    cv2.imshow("Image", img) # feed outwards
      
    # tackle exit using q
    key = cv2.waitKey(1)&0xFF
    if key == ord('q'):
        break
        
    # wait 1 millisecond before next frame
    cv2.waitKey(1) 
    
# release control of camera
camera.release()
# destroy all window under operations
cv2.destroyAllWindows()